# Content beased filtering using BERT
Recommendations using BERT

In [35]:
import pandas as pd
import numpy as np

import re
import string

from nltk.classify.textcat import TextCat

import dask.dataframe as dd
import multiprocessing

from keybert import KeyBERT

from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import altair as alt
alt.renderers.enable('mimetype')

from sklearn.metrics.pairwise import cosine_similarity

### Data Cleaning

In [36]:
path='./Data/'

df_books1=pd.read_csv(path +'Books-old.csv') 
df_books2 = pd.read_csv(path + 'books_1.Best_Books_Ever.csv') #To skip bad lines
# df_books2 = pd.read_csv(path + 'books-goodreads.csv', on_bad_lines='skip', dtype=str, low_memory=False)
df_ratings=pd.read_csv(path + 'Ratings.csv') 
df_users=pd.read_csv(path + 'Users.csv') 

#Renameing
df_books2.rename(columns={'title':'Book-Title'}, inplace=True)


C:\Users\aksel\AppData\Local\Temp\ipykernel_34768\3939204259.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books1=pd.read_csv(path +'Books-old.csv')


In [37]:
print("df_books1:")
print(f"Shape: {df_books1.shape}")
print(f"Columns: {df_books1.columns.tolist()}\n")

print("df_books2:")
print(f"Shape: {df_books2.shape}")
print(f"Columns: {df_books2.columns.tolist()}")

df_books1:
Shape: (271360, 8)
Columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']

df_books2:
Shape: (52478, 25)
Columns: ['bookId', 'Book-Title', 'series', 'author', 'rating', 'description', 'language', 'isbn', 'genres', 'characters', 'bookFormat', 'edition', 'pages', 'publisher', 'publishDate', 'firstPublishDate', 'awards', 'numRatings', 'ratingsByStars', 'likedPercent', 'setting', 'coverImg', 'bbeScore', 'bbeVotes', 'price']


In [38]:
# Merged dataset



merged_book_df=pd.merge(df_books1, df_books2, on='Book-Title', how='inner') 

print("df_books_merged:")
print(f"Shape: {merged_book_df.shape}")
print(f"Columns: {merged_book_df.columns.tolist()}")



merged_book_df.dropna(subset=["description"], inplace=True)

print("df_books_merged:")
print(f"Shape: {merged_book_df.shape}")
print(f"Columns: {merged_book_df.columns.tolist()}")

merged_book_df.head


df_books_merged:
Shape: (27191, 32)
Columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'bookId', 'series', 'author', 'rating', 'description', 'language', 'isbn', 'genres', 'characters', 'bookFormat', 'edition', 'pages', 'publisher', 'publishDate', 'firstPublishDate', 'awards', 'numRatings', 'ratingsByStars', 'likedPercent', 'setting', 'coverImg', 'bbeScore', 'bbeVotes', 'price']
df_books_merged:
Shape: (26965, 32)
Columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'bookId', 'series', 'author', 'rating', 'description', 'language', 'isbn', 'genres', 'characters', 'bookFormat', 'edition', 'pages', 'publisher', 'publishDate', 'firstPublishDate', 'awards', 'numRatings', 'ratingsByStars', 'likedPercent', 'setting', 'coverImg', 'bbeScore', 'bbeVotes', 'price']


<bound method NDFrame.head of              ISBN              Book-Title            Book-Author  \
0      0399135782  The Kitchen God's Wife                Amy Tan   
1      080410753X  The Kitchen God's Wife                Amy Tan   
2      080410753x  The Kitchen God's Wife                Amy Tan   
3      1590400356  The Kitchen God's Wife                Amy Tan   
4      1558002669  The Kitchen God's Wife                Amy Tan   
...           ...                     ...                    ...   
27186  0441702902                Radiance           Anne Maybury   
27187  0441702902                Radiance           Anne Maybury   
27188  0441702902                Radiance           Anne Maybury   
27189  0345298608        The Broken Sword       Northam Anderson   
27190  0451521390              Pathfinder  James Fenimore Cooper   

      Year-Of-Publication         Publisher  \
0                    1991  Putnam Pub Group   
1                    1992         Ivy Books   
2           

## Data cleanup

In [39]:
merged_book_df['description'] = merged_book_df['description'].str.replace(r'@\w+', '<user>', regex=True)

merged_book_df['description'] = merged_book_df['description'].str.replace(r'http\w+', '<url>', regex=True)

#list(merged_book_df.description[merged_book_df.Id == 1099555]) #Description with url and html tag

merged_book_df['description'] = merged_book_df['description'].str.replace(r'[{}]+'.format(string.punctuation), '', regex=True)


print(merged_book_df['description'].head())


0    Winnie and Helen have kept each others worst s...
1    Winnie and Helen have kept each others worst s...
2    Winnie and Helen have kept each others worst s...
3    Winnie and Helen have kept each others worst s...
4    Winnie and Helen have kept each others worst s...
Name: description, dtype: object


In [80]:
from tqdm import tqdm


model=KeyBERT()

def get_keywords_from_description(description):
    keywords = model.extract_keywords(description, keyphrase_ngram_range=(1, 1), stop_words="english")
    keywords = " ".join([k[0] for k in keywords])
    return keywords

get_keywords_from_description(merged_book_df['description'].iloc[20])

'machiavellian machiavelli prince noble mercenary'

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


duplicates = merged_book_df[merged_book_df.duplicated(subset=['Book-Title', 'description'], keep=False)]
print(duplicates)

# Remove duplicates
merged_book_df = merged_book_df.drop_duplicates(subset=['Book-Title', 'description'])
print(f"Shape: {merged_book_df.shape}")

# make a csv file with the data
merged_book_df.to_csv('Data/merged_books.csv', index=False)


# Assuming merged_book_df is your dataframe with book descriptions
descriptions = merged_book_df['description'].fillna('')

#print one title in merged_book_df
print(merged_book_df['Book-Title'].iloc[0])

# Step 3: Compute the TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(descriptions)

# Step 4: Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

Empty DataFrame
Columns: [ISBN, Book-Title, Book-Author, Year-Of-Publication, Publisher, Image-URL-S, Image-URL-M, Image-URL-L, bookId, series, author, rating, description, language, isbn, genres, characters, bookFormat, edition, pages, publisher, publishDate, firstPublishDate, awards, numRatings, ratingsByStars, likedPercent, setting, coverImg, bbeScore, bbeVotes, price]
Index: []

[0 rows x 32 columns]
Shape: (11388, 32)
The Kitchen God's Wife


In [73]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the book that matches the title
    idx = merged_book_df[merged_book_df['Book-Title'] == title].index[0]

    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:11]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]
    # print(merged_book_df['description'].iloc[book_indices])
    
    # Return the top 10 most similar books
    return merged_book_df['Book-Title'].iloc[book_indices]

# Example usage
print(get_recommendations("Voices"))

6803              Sense of Evil
1824                     Chains
6988                   Ricochet
9839             One for Sorrow
5568     The Portrait of a Lady
1300                   The Heir
9635           Keeping the Moon
4905                       Lies
11209                Wide Awake
11135       The Collected Poems
Name: Book-Title, dtype: object
